# 3. Sweep 사용해보기

sweep은 간단하게 말하면 지정한 hyperparameter를 자동으로 최적화시켜줍니다.

gird search, 혹은 random search를 위한 도구로 이해하시면 좋을 것 같습니다.

**1. Baseline 살펴보기**, **2. WanDB 적용하기**와 겹치는 부분은 생략하도록 하겠습니다.

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 12.6 MB/s 
     |████████████████████████████████| 181 kB 52.4 MB/s 
     |████████████████████████████████| 156 kB 53.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0e0974387e36387525e544e91094e1dea976e23ceb67afe8c0b2dd8f88c0ed90
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

In [ ]:
# 커맨드 라인을 통한 로그인 방식

!wandb login e5bcc5be2415513ec9c1ff6d0938e9cb06378019 # 이와 같이 미리 토큰 값을 입력할 수 있습니다.자세한 옵션은 아래 링크 참조

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# 코드쉘을 이용한 방식 이 방법을 추천

# wandb.login()

In [ ]:
# Data load

cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
CLASS_NAMES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "hores", "ship", "truck"]

In [ ]:
# Normalization

X_train, X_test = X_train / 255., X_test / 255.

In [ ]:
# Validation Set

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25)

In [ ]:
# samples for wandb

val_images, val_labels = X_test[:32], y_test[:32]

## 1. Sweep Config 생성하기

**2. WanDB 적용하기**에서 config 작성한 것과 비슷한 요령으로 작성하면 됩니다.

다만 조절을 원하는 hyperparameter 혹은 기록해두고 싶은 parameter 이외에도 입력해야하는 부분이 있습니다.

[python code](https://docs.wandb.ai/guides/sweeps/configuration)

In [ ]:
sweep_config = {
    "name": "my-test-sweep",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "optimizer" : {
            'value' : 'adam'
            },
        "batch_size" : {
            "values" : [32, 64]
            },
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epochs" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

### default config에 대해서

아래서 소개할 sweep method는 train 함수를 필요로 합니다.

모델 생성 등에 sweep_config를 사용하기 어렵기 때문에 default가 될 값이 필요합니다.

default config에 있는 값 중 sweep_config에서 지정한 값은 sweep_config 값을 따르고 나머지는 default 값을 사용합니다.

In [ ]:
def train():
    default_config = {
        "n_channel_1" : 32,
        "n_channel_2" : 64,
        "n_dense" : 1024,
        "learning_rate" : 0.05,
        "epochs" : 5,
        "batch_size" : 32,
        "optimizer" : 'adam',
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"]
    }

    wandb.init(config = default_config)
    config = wandb.config

    # Model

    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(config.n_channel_1, (3,3), activation='relu', input_shape=(32,32,3)))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(config.n_channel_2, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(config.n_dense, activation='relu'))
    model.add(keras.layers.Dense(len(CLASS_NAMES), activation='softmax'))

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우 아래 코드를 응용할 것!

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    model.fit(X_train, y_train,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback(data_type = 'image',
                                         training_data = (val_images, val_labels),
                                         labels = CLASS_NAMES)])
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})

## 2. wandb.sweep()과 wandb.agent()

위에서 설정한 sweep_config 값을 통해 sweep id를 생성하고 wandb의 agent에 해당 아이디와 실행시킬 코드를 전달하면 agent가 sweep id에 지시된대로 experiment를 시행해 optimization을 진행합니다.

wandb.sweep()

[python](https://docs.wandb.ai/ref/python/sweep)

[github](https://github.com/wandb/wandb/blob/latest/wandb/sdk/wandb_sweep.py#L31-L111)

wandb.agent()

[python](https://docs.wandb.ai/ref/python/agent)

[github](https://github.com/wandb/wandb/blob/latest/wandb/wandb_agent.py#L526-L587)

### entity와 project에 대해서

entitiy와 project는 agent뿐만 아니라 wandb.sweep에서도 설정할 수 있고 그 이전에 sweep_config에서도 설정할 수도 있습니다.

저는 최종단계인 agent에 입력했지만 원하는대로 커스텀하시기 바랍니다.

단, entitiy와 project를 입력하는 것 자체는 꼭 기억하시기 바랍니다.

In [ ]:
sweep_id = wandb.sweep(sweep_config)

# run the sweep
wandb.agent(sweep_id,
            function=train,
            entity = 'hongdune',
            project = 'keras-example')

Create sweep with ID: c92ovj21
Sweep URL: https://wandb.ai/hongdune/keras-example/sweeps/c92ovj21


wandb: Agent Starting Run: nl964r1b with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0878608752956713
wandb: 	optimizer: adam
wandb: Currently logged in as: hongdune. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/5
1172/1172 [==============================] - ETA: 0s - loss: 8.6295 - accuracy: 0.1015INFO:tensorflow:Assets written to: /content/wandb/run-20220803_212328-nl964r1b/files/model-best/assets


wandb: Adding directory to artifact (/content/wandb/run-20220803_212328-nl964r1b/files/model-best)... Done. 0.5s


...

wandb: Agent Starting Run: dijtwgw2 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 6
wandb: 	learning_rate: 0.03583200383349411
wandb: 	optimizer: adam


Epoch 1/6
1170/1172 [============================>.] - ETA: 0s - loss: 3.0941 - accuracy: 0.1031INFO:tensorflow:Assets written to: /content/wandb/run-20220803_234325-dijtwgw2/files/model-best/assets


wandb: Adding directory to artifact (/content/wandb/run-20220803_234325-dijtwgw2/files/model-best)... Done. 0.4s


1172/1172 [==============================] - 12s 10ms/step - loss: 3.0929 - accuracy: 0.1032 - val_loss: 2.3057 - val_accuracy: 0.0994 - _timestamp: 1659570221.2714 - _runtime: 15.4526
Epoch 2/6
 199/1172 [====>.........................] - ETA: 5s - loss: 2.3062 - accuracy: 0.1036

wandb: Ctrl + C detected. Stopping sweep.


## 3. 종료하기

경우에 따라(parameter의 조합이 무한대일때) sweep이 종료가 되지 않을 수 있습니다.

이 경우 ctrl + c 로(여기에선 실행 중단)으로 종료해주셔야합니다.

In [ ]:
wandb.finish()